# how to run

## step1
At first, please run 
```
$ python split_dfset.py
```
which will cause errors, but no problem if folders 0001/ to 0006/ are made.

## step2 
Please find the cell containing previous_it_str and it_str and 
change the lines to 
```
previous_it_str = '0000'
it_str = '0001'
```
and "Run"->"Run All Cells". 
Files will be made in 0001/. 

0001/free_energy_history.txt is the important history file.

```
'unstable': False,
'converged': False,
```
in phase1_result["results"].get_dict() will be shown.
The free energy isn't converged. 

## step 3
Change the lines to 
```
previous_it_str = '0001'
it_str = '0002'
```
and "Run"->"Run All Cells". 
Files will be made in 0002/. 

0002/free_energy_history.txt is the important history file.

```
'unstable': False,
'converged': True,
```
in phase1_result["results"].get_dict() will be shown.

The free energy is converged now, and pdf files are also made in the 0002/ folder. 

In [49]:
import argparse
import os
import shutil
from pathlib import Path

import numpy as np
from ase.units import Ry

from aiida_alamode.workflows import PhononCalculator
from aiida_alamode.workflows.model_selection import ForceConstantModelSelector
from aiida_alamode.io import DfHistoryData
from io import TextIOWrapper


In [50]:
import aiida
from aiida.plugins import DataFactory, WorkflowFactory
from aiida.plugins.factories import CalculationFactory

from aiida.orm import Str, Int, Dict, List, Float, load_node
from aiida.engine import calcfunction, workfunction, submit, run, run_get_node
from aiida.engine import WorkChain, ToContext, if_, append_
from aiida_alamode.tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os

from aiida.orm import Code

# must execute it in the first place
aiida.load_profile()


In [51]:
# make computer local
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')

In [52]:
SinglefileData = DataFactory('singlefile')
StructureData = DataFactory('structure')
ArrayData = DataFactory('array')
FrameData = DataFactory('dataframe.frame')


In [53]:
code_anphon_calculator = get_or_create_code('alamode.anphon_calculator',
                          computer_local,
                          '/home/max/Documents/alamode/bin/anphon')

# code_anphon_calculator = CalculationFactory('alamode.anphon_calculator') # doesn't work.

type(code_anphon_calculator)

aiida.orm.nodes.data.code.Code

In [54]:
basedir = os.getcwd()
print("basedir", basedir)

poscar_file =  os.path.join(basedir, "SPOSCAR")
prim_file = os.path.join(basedir, "relaxed_final_symm.POSCAR.vasp")
previous_it_str = '0000'
it_str = '0001'
cwd = os.path.join(basedir, it_str)

dfset_file = os.path.join(cwd, "dfset")
history_file = os.path.join(basedir, previous_it_str, "free_energy_history.txt")
cwd

basedir /home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/converged,run


'/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/converged,run/0002'

In [55]:
from ase import io
superatoms = io.read(poscar_file, format="vasp")
superstructure_node = StructureData(ase=superatoms).store()
primatoms = io.read(prim_file, format="vasp")
primstructure_node = StructureData(ase=primatoms).store()


In [56]:
dfset = np.loadtxt(dfset_file)
numatoms = len(superatoms.numbers)
dfset = dfset.reshape(-1, numatoms, 6)
dfset_node = ArrayData()
dfset_node.set_array('dfset',dfset)
dfset_node.store()


<ArrayData: uuid: 104f8bb3-355c-4c44-91c0-febad0fe17b4 (pk: 4919)>

In [57]:
dfset.shape

(4, 150, 6)

In [58]:
history = DfHistoryData().from_file(history_file)
history_node = FrameData(df=history.df)
history_node.df

,Iteration,No. training data,best cuoff [bohr],lowest omega,ratio imaginary,Fvib (mev/atom) 0,Fvib (mev/atom) 1,Fvib (mev/atom) 2,Fvib (mev/atom) 3,Fvib (mev/atom) 4,...,Fvib (mev/atom) 11,Fvib (mev/atom) 12,Fvib (mev/atom) 13,Fvib (mev/atom) 14,Fvib (mev/atom) 15,Fvib (mev/atom) 16,Fvib (mev/atom) 17,Fvib (mev/atom) 18,Fvib (mev/atom) 19,Fvib (mev/atom) 20
0,1,2,22.161871,0.0,0.0,0.22243,0.079797,-0.358955,-0.969234,-1.694341,...,-8.511323,-9.645393,-10.808181,-11.997491,-13.211409,-14.448312,-15.706757,-16.985474,-18.283339,-19.599336


# run a step

In [59]:
cwd

'/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/converged,run/0002'

In [60]:
checkConvergenceFreeenergy_workchain = WorkflowFactory('alamode.check_convergence_freeenergy')

def phaseall(superstructure_node, primstructure_node, dfset_node, history_node, anphon_code, cwd=None,
            qspacing=0.1, conv_thr=0.1, temp=1000, cv=2, ninc=2, verbosity=1):   
    # params
    params = {"qspacing": qspacing, "conv_thr": conv_thr, "temp": temp, "cv": cv,
              "ninc": ninc, "verbosity": verbosity}
   
    inputs = {'super_structure': superstructure_node,   
              'primitive_structure': primstructure_node,
              'dfset': dfset_node,
              'params': Dict(dict=params),
              'history': history_node,
             'anphon_code': anphon_code}
    if cwd is not None:
        inputs.update({'cwd': Str(cwd)})

    result = run(checkConvergenceFreeenergy_workchain, **inputs)
    
    return result

phase1_result = phaseall(superstructure_node, primstructure_node, dfset_node, history_node, code_anphon_calculator, cwd)


Iter #2: Fvib (meV/atom) = -7.408, Delta_Fvib (meV/atom) = 0.000
Convergence criterion of |Delta_Fvib| < 0.100 has been satisfied!


findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Times New Roman'] not found. Falling back to DejaVu Sans.


In [61]:
history_file = os.path.join(cwd,  "free_energy_history.txt")

print(history_file)
df = phase1_result['history'].df
df.to_csv(history_file, index=False)

/home/max/Documents/aiida-alamode_public/example_dxmag/free_energy_convergence/converged,run/0002/free_energy_history.txt


In [62]:
phase1_result

{'rc_best': <Float: uuid: a5e081dc-5f09-498a-a645-e15929c37764 (pk: 4924) value: 25.161871374251>,
 'fc': <ArrayData: uuid: f72c50d9-b177-47db-85bf-29c54beb650c (pk: 4925)>,
 'ols_history': <PandasFrameData: uuid: 66eae01d-fde8-459d-abd2-4494073f93ba (pk: 4926)>,
 'band_results': <Dict: uuid: 769be54a-5426-47cb-8e66-015c0a3e90c2 (pk: 4937)>,
 'bands': <SinglefileData: uuid: 031a4f85-42dd-450c-86d5-ef3062f61292 (pk: 4936)>,
 'thermo_results': <Dict: uuid: 01144e6c-4792-4e84-b937-2db3fd268c4f (pk: 4947)>,
 'dosthermo_properties': <ArrayData: uuid: e3c3cc05-1d6a-4660-af20-e796b203c7ef (pk: 4946)>,
 'results': <Dict: uuid: 2e1631cd-3506-43e1-86a1-b649174c6119 (pk: 4948)>,
 'history': <PandasFrameData: uuid: d189dfe4-993b-4b0b-8212-df985c922659 (pk: 4949)>,
 'img_files': <Dict: uuid: a2517986-0225-46df-98fd-1a28bed6ac04 (pk: 4956)>}

In [63]:
phase1_result["history"].df

,Iteration,No. training data,best cuoff [bohr],lowest omega,ratio imaginary,Fvib (mev/atom) 0,Fvib (mev/atom) 1,Fvib (mev/atom) 2,Fvib (mev/atom) 3,Fvib (mev/atom) 4,...,Fvib (mev/atom) 11,Fvib (mev/atom) 12,Fvib (mev/atom) 13,Fvib (mev/atom) 14,Fvib (mev/atom) 15,Fvib (mev/atom) 16,Fvib (mev/atom) 17,Fvib (mev/atom) 18,Fvib (mev/atom) 19,Fvib (mev/atom) 20
0,1.0,2.0,22.161871,0.0,0.0,0.222430,0.079797,-0.358955,-0.969234,-1.694341,...,-8.511323,-9.645393,-10.808181,-11.997491,-13.211409,-14.448312,-15.706757,-16.985474,-18.283339,-19.599336
1,2.0,4.0,25.161871,0.0,0.0,0.222448,0.079820,-0.358918,-0.969182,-1.694271,...,-8.511137,-9.645194,-10.807963,-11.997255,-13.211155,-14.448040,-15.706466,-16.985166,-18.283013,-19.599001


In [64]:
phase1_result["results"].get_dict()

{'cutoffs': [22.161871374251],
 'unstable': False,
 'converged': True,
 'ninc_used': 2,
 'iter_start': 1,
 'exist_history': True,
 'lowest_omegas': [0.0],
 'previous_fvib': -7.4085873787796,
 'imaginary_ratios': [0.0]}